<a href="https://colab.research.google.com/github/jeffersonramelo/Textual-Analysis/blob/main/C3_Sentimento_FINBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# Caminho do arquivo CSV
caminho_arquivo = "/content/C123_dataset sentimento rotulados.csv"

# Ler o arquivo CSV
import pandas as pd
df = pd.read_csv(caminho_arquivo, sep=",", encoding='latin-1')


In [ ]:
df.head()

,sentimento_rotulado,texto,id
0,neutral,"According to Gran , the company has no plans t...",1
1,neutral,Technopolis plans to develop in stages an area...,2
2,negative,The international electronic industry company ...,3
3,positive,With the new production plant the company woul...,4
4,positive,According to the company 's updated strategy f...,5


In [ ]:
# Acessar os textos contidos nas celulas, só substituir o número da celula que deseja vizualizar
celula = df.loc[10, 'texto']

# Imprimir o conteúdo da célula
print(celula)

TeliaSonera TLSN said the offer is in line with its strategy to increase its ownership in core business holdings and would strengthen Eesti Telekom 's offering to its customers .


In [2]:
textos = df["texto"].tolist()

In [3]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.0 MB/s eta 0:00:00


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [5]:
# Carregar o modelo FinBERT pré-treinado
model_name = "ProsusAI/finbert"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Pré-processamento e análise de sentimento
sentimentos = []

for texto in df['texto']:
    encoded_input = tokenizer(texto, padding=True, truncation=True, max_length=512, return_tensors="pt")
    output = model(**encoded_input)
    logits = output.logits
    predicted_label = torch.argmax(logits).item()

    # Mapear os valores de sentimento para rótulos correspondentes
    if predicted_label == 0:
        sentimento = "negative"
    elif predicted_label == 1:
        sentimento = "positive"
    else:
        sentimento = "neutral"

    sentimentos.append(sentimento)

# Criar nova coluna com a classificação de sentimento do FinBERT
df["sentimentoFINBERT"] = sentimentos


In [6]:
# Visualizar os resultados
df.head()

,sentimento_rotulado,texto,id,sentimentoFINBERT
0,neutral,"According to Gran , the company has no plans t...",1,neutral
1,neutral,Technopolis plans to develop in stages an area...,2,neutral
2,negative,The international electronic industry company ...,3,positive
3,positive,With the new production plant the company woul...,4,negative
4,positive,According to the company 's updated strategy f...,5,negative


In [7]:
# Contar a frequência de cada valor na coluna "sentimentoFINBERT"
contagem_sentimentos = df["sentimentoFINBERT"].value_counts()

# Exibir os resultados
print(contagem_sentimentos)


neutral     2567
negative    1549
positive     730
Name: sentimentoFINBERT, dtype: int64


In [8]:
# Verificar a precisão do modelo sentimentoFINBERT em relação a sentimento_rotulado
corretos = sum(df['sentimento_rotulado'] == df['sentimentoFINBERT'])
total = len(df)

precisao = corretos / total
precisao


0.5156830375567478

In [ ]:
import pandas as pd

# Isolar as variáveis "ata" e "sentimentoFINBERT"
dados_isolados = df[["id", "sentimentoFINBERT"]]

# Salvar os dados isolados em um arquivo Excel (.xlsx)
dados_isolados.to_excel("/content/sentimentoFINBERT.xlsx", index=False)
